This notebook is not used operationally or for any validation, its only purpose is to have a clear understanding of the core functions of the AA workflow. The outputs and dimensions of each main step can thus be identified here.

**Import required libraries and functions**

In [1]:
%cd ..

/home/u/amine.barkaoui/anticipatory-action


In [2]:
import os
import datetime
import pandas as pd

from config.params import Params

from AA.helper_fns import (
    read_forecasts,
    read_observations,
    compute_district_average,
    merge_un_biased_probs,
    merge_probabilities_triggers_dashboard,
)

from hip.analysis.analyses.drought import (
    get_accumulation_periods,
    run_accumulation_index,
    run_gamma_standardization,
    run_bias_correction,
    compute_probabilities,
)

from hip.analysis.aoi.analysis_area import AnalysisArea

In [3]:
from hip.analysis.compute.utils import start_dask
client = start_dask(n_workers=1)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43179
INFO:distributed.scheduler:  dashboard at:  /user-redirect/proxy/8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39573'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:46833', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46833
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55170
INFO:distributed.scheduler:Receive client connection: Client-b717cf8d-112f-11f0-814f-0a5d9b3b8993
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55182
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.

**Define parameters**

The `config/{country}_config.yaml` file gathers all the parameters used in the operational script and that can be customized. For example, the *monitoring_year*, the list of districts or the intensity levels can be defined in that file.

In [67]:
params = Params(iso="MWI", issue=11, index="SPI")

**Read shapefile**

In [5]:
# Define aoi to read datasets using hip-analysis
area = AnalysisArea.from_admin_boundaries(
    iso3=params.iso.upper(),
    admin_level=2,
    resolution=0.25,
    datetime_range=f"1981-01-01/{params.monitoring_year + 1}-06-30",
)

# Read the shapefile
gdf = area.get_dataset([area.BASE_AREA_DATASET])
#gdf

**Read forecasts**

In [68]:
# When update is set to False, the downscaled dataset is read from a local folder or a s3 bucket. Otherwise, it is directly read from HDC.
forecasts = read_forecasts(
    area,
    params.issue,
    f"{params.data_path}/data/{params.iso}/zarr/2022/{str(params.issue).zfill(2)}/forecasts.zarr",
    update=False,  # True,
)
forecasts

<xarray.DataArray 'tp' (time: 8702, ensemble: 51, latitude: 32, longitude: 14)> Size: 795MB
dask.array<open_dataset-tp, shape=(8702, 51, 32, 14), dtype=float32, chunksize=(8702, 1, 32, 14), chunktype=numpy.ndarray>
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    issue        (time) <U7 244kB dask.array<chunksize=(8702,), meta=np.ndarray>
  * latitude     (latitude) float64 256B -9.375 -9.625 -9.875 ... -16.88 -17.12
  * longitude    (longitude) float64 112B 32.62 32.88 33.12 ... 35.62 35.88
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 70kB 1981-11-01 1981-11-02 ... 2022-05-31
Attributes:
    nodata:   nan
    units:    mm

**Read observations**

In [6]:
# Observations data reading (already stored as the dataset used is the same as the one used in the pre-season/analytical script)
observations = read_observations(
    area,
    f"{params.data_path}/data/{params.iso}/zarr/{params.calibration_year}/obs/observations.zarr",
)
observations

<xarray.DataArray 'band' (time: 15156, latitude: 32, longitude: 14)> Size: 54MB
dask.array<open_dataset-band, shape=(15156, 32, 14), dtype=float64, chunksize=(1, 32, 14), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 256B -9.375 -9.625 -9.875 ... -16.88 -17.12
  * longitude    (longitude) float64 112B 32.62 32.88 33.12 ... 35.62 35.88
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 121kB 1981-01-01 ... 2022-06-30
Attributes:
    nodata:   nan

**Read pre-computed triggers**

Now that we got all the data we need, let's read the triggers file so we can merge the probabilities with it once we have them.

In [ ]:
# Read triggers file
if os.path.exists(
    f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv"
):
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv",
    )
else:
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/triggers/triggers.spi.dryspell.{params.calibration_year}.pilots.csv",
    )
triggers_df

**Get accumulation periods covered by the forecasts of the defined issue month**

In [69]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts,
    params.start_season,
    params.end_season,
    params.min_index_period,
    params.max_index_period,
)
accumulation_periods

{'JFM': (1, 2, 3),
 'FMA': (2, 3, 4),
 'MAM': (3, 4, 5),
 'NDJ': (11, 12, 1),
 'DJF': (12, 1, 2)}

Here we focus on the pipeline for one indicator (one period) so we select a single element from the above dictionary (November-December using October forecasts).

In [70]:
# Get single use case
period_name, period_months = list(accumulation_periods.items())[0]  # [4]
period_name, period_months

('JFM', (1, 2, 3))

**Run accumulation (sum for SPI)**

In [71]:
# Remove 1980 season to harmonize datasets between different indexes
forecasts = forecasts.where(
    forecasts.time.dt.date >= datetime.date(1981, params.start_season, 1), drop=True
)
observations = observations.where(
    observations.time.dt.date >= datetime.date(1981, params.start_season, 1), drop=True
)

In [72]:
# Accumulation
accumulation_fc = run_accumulation_index(
    forecasts.chunk(dict(time=-1)),
    params.aggregate,
    period_months,
    (params.start_season, params.end_season),
    forecasts=True,
)
accumulation_obs = run_accumulation_index(
    observations.chunk(dict(time=-1)),
    params.aggregate,
    period_months,
    (params.start_season, params.end_season),
)

In [77]:
obs_district = compute_district_average(accumulation_obs, area)

/envs/shared/hdc/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 9.86 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [80]:
obs_district.sel(district='Mulanje').to_pandas().to_csv("sample_mwi_chirps_mulanje_jfm.csv")

In [75]:
fc_district = compute_district_average(accumulation_fc, area)

In [79]:
fc_district.sel(district='Mulanje').to_pandas().to_csv("sample_mwi_ecmwf_11_mulanje_jfm.csv")

In [73]:
period_months

(1, 2, 3)

In [74]:
params.issue

11

**Run standardization (SPI)**

In [25]:
# Anomaly
anomaly_fc = run_gamma_standardization(
    accumulation_fc.load(),
    params.hist_anomaly_start,
    params.hist_anomaly_stop,
    members=True,
)
#anomaly_obs = run_gamma_standardization(
#    accumulation_obs.load(),
#    params.hist_anomaly_start,
#    params.hist_anomaly_stop,
#)

/envs/shared/hdc/lib/python3.10/site-packages/hip/analysis/analyses/drought.py:237: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['date', 'ensemble'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'date', 'ensemble'])` before assigning new coordinate values.
  rfh["time"] = [


**Run bias correction**

In [12]:
# Bias correction
index_bc = run_bias_correction(
    anomaly_fc,
    anomaly_obs,
    start_monitoring=params.start_monitoring,
    year=params.monitoring_year,
    issue=int(params.issue),
    nearest_neighbours=8,
    enso=True,
)

/envs/shared/hdc/lib/python3.10/site-packages/xclim/sdba/utils.py:354: RuntimeWarning: All-nan slice encountered in interp_on_quantiles
  warn(


**Run probabilities**

In [34]:
# Change dryspell sign as we compare values to a negative threshold to get probabilities
if params.index == "dryspell":
    anomaly_fc *= -1
    index_bc *= -1
    anomaly_obs *= -1

In [37]:
# Probabilities without Bias Correction
probabilities = (
    anomaly_fc.groupby(anomaly_fc.time.dt.year)
    .apply(
        compute_probabilities,
        levels=params.intensity_thresholds,
    )
    .round(2)
)

In [40]:
anomaly_fc.where(anomaly_fc.time.dt.year == 1981, drop=True).isel(longitude=0, latitude=0)

<xarray.DataArray 'tp' (time: 25)> Size: 200B
array([ -714.,  1696.,  1108.,  1096.,   217.,   968.,    60.,    63.,
       -1236.,   758.,  -890.,   545.,  -615.,  -483.,  -231.,  1390.,
        -352.,   362.,  -504.,    10.,  1319.,  -638.,  -490.,    -5.,
       -2603.])
Coordinates:
    latitude     float64 8B -8.125
    longitude    float64 8B 22.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 200B 1981-01-01 1981-01-02 ... 1981-01-25
Attributes:
    nodata:                 nan
    units:                  mm
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-02-20 00:00:00

In [36]:
probs_district = compute_district_average(probabilities, area)

In [40]:
import xarray as xr
check = xr.open_zarr(f"{params.data_path}/data/zmb/zarr/2022/01/spi FMA/probabilities.zarr").tp

In [41]:
check.sel(district='Zimba', category='Severe').load()

<xarray.DataArray 'tp' (year: 41)> Size: 328B
array([0.24  , 0.27  , 0.08  , 0.115 , 0.33  , 0.275 , 0.22  , 0.09  ,
       0.215 , 0.055 , 0.275 , 0.175 , 0.16  , 0.27  , 0.105 , 0.195 ,
       0.49  , 0.    , 0.08  , 0.08  , 0.1   , 0.255 , 0.085 , 0.205 ,
       0.04  , 0.16  , 0.    , 0.2   , 0.11  , 0.    , 0.2   , 0.135 ,
       0.12  , 0.14  , 0.325 , 0.12  , 0.095 , 0.2325, 0.17  , 0.2025,
       0.1975])
Coordinates:
    category     <U8 32B 'Severe'
    district     <U13 52B 'Zimba'
    spatial_ref  int64 8B 0
  * year         (year) int64 328B 1981 1982 1983 1984 ... 2018 2019 2020 2021

In [42]:
probs_district.sel(district='Zimba', category='Severe')

<xarray.DataArray 'mean' (time: 41)> Size: 328B
array([0.24  , 0.27  , 0.08  , 0.115 , 0.33  , 0.275 , 0.22  , 0.09  ,
       0.215 , 0.055 , 0.275 , 0.175 , 0.16  , 0.27  , 0.105 , 0.195 ,
       0.49  , 0.    , 0.08  , 0.08  , 0.1   , 0.255 , 0.085 , 0.205 ,
       0.04  , 0.16  , 0.    , 0.2   , 0.11  , 0.    , 0.2   , 0.135 ,
       0.12  , 0.14  , 0.325 , 0.12  , 0.095 , 0.2325, 0.17  , 0.2025,
       0.1975])
Coordinates:
  * time      (time) int64 328B 1981 1982 1983 1984 1985 ... 2018 2019 2020 2021
    category  <U6 24B 'Severe'
    district  <U13 52B 'Zimba'

In [16]:
# Probabilities without Bias Correction
probabilities = compute_probabilities(
    anomaly_fc.where(anomaly_fc.time.dt.year == params.monitoring_year, drop=True),
    levels=params.intensity_thresholds,
).round(2)
display(probabilities)

<xarray.DataArray 'tp' (category: 3, latitude: 67, longitude: 44)> Size: 71kB
array([[[0.2 , 0.2 , 0.2 , ..., 0.24, 0.24, 0.22],
        [0.2 , 0.18, 0.18, ..., 0.2 , 0.24, 0.22],
        [0.18, 0.18, 0.18, ..., 0.22, 0.24, 0.2 ],
        ...,
        [0.18, 0.14, 0.18, ..., 0.25, 0.25, 0.27],
        [0.18, 0.14, 0.16, ..., 0.24, 0.25, 0.27],
        [0.16, 0.14, 0.16, ..., 0.22, 0.25, 0.27]],

       [[0.22, 0.22, 0.22, ..., 0.27, 0.25, 0.27],
        [0.24, 0.2 , 0.2 , ..., 0.27, 0.25, 0.27],
        [0.22, 0.22, 0.2 , ..., 0.25, 0.25, 0.25],
        ...,
        [0.22, 0.22, 0.2 , ..., 0.31, 0.29, 0.29],
        [0.22, 0.18, 0.18, ..., 0.33, 0.33, 0.29],
        [0.18, 0.18, 0.18, ..., 0.31, 0.31, 0.29]],

       [[0.25, 0.27, 0.25, ..., 0.31, 0.33, 0.31],
        [0.25, 0.27, 0.27, ..., 0.33, 0.35, 0.33],
        [0.27, 0.29, 0.29, ..., 0.35, 0.35, 0.33],
        ...,
        [0.25, 0.29, 0.27, ..., 0.35, 0.35, 0.37],
        [0.24, 0.27, 0.25, ..., 0.33, 0.33, 0.35],
        [0.24, 0.22, 0.27, ..., 0.35, 0.33, 0.33]]])
Coordinates:
  * latitude     (latitude) float64 536B -10.38 -10.62 -10.88 ... -26.62 -26.88
  * longitude    (longitude) float64 352B 30.12 30.38 30.62 ... 40.62 40.88
    spatial_ref  int32 4B 4326
  * category     (category) object 24B 'Severe' 'Moderate' 'Mild'

In [17]:
# Probabilities after Bias Correction
probabilities_bc = compute_probabilities(
    index_bc, levels=params.intensity_thresholds
).round(2)
display(probabilities_bc)

<xarray.DataArray 'scen' (category: 3, longitude: 44, latitude: 67)> Size: 71kB
array([[[0.08, 0.04, 0.  , ..., 0.06, 0.06, 0.06],
        [0.08, 0.04, 0.  , ..., 0.06, 0.06, 0.08],
        [0.08, 0.04, 0.02, ..., 0.1 , 0.1 , 0.08],
        ...,
        [0.14, 0.14, 0.18, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.08, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]],

       [[0.22, 0.2 , 0.08, ..., 0.08, 0.1 , 0.1 ],
        [0.22, 0.2 , 0.1 , ..., 0.14, 0.1 , 0.08],
        [0.22, 0.18, 0.1 , ..., 0.12, 0.14, 0.1 ],
        ...,
        [0.14, 0.14, 0.22, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.08, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]],

       [[0.31, 0.37, 0.27, ..., 0.16, 0.12, 0.12],
        [0.33, 0.35, 0.31, ..., 0.14, 0.14, 0.14],
        [0.29, 0.29, 0.31, ..., 0.2 , 0.18, 0.16],
        ...,
        [0.14, 0.16, 0.22, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.1 , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]]])
Coordinates:
  * latitude     (latitude) float64 536B -10.38 -10.62 -10.88 ... -26.62 -26.88
  * longitude    (longitude) float64 352B 30.12 30.38 30.62 ... 40.62 40.88
    spatial_ref  int32 4B 4326
  * category     (category) object 24B 'Severe' 'Moderate' 'Mild'

**Admin-2 level aggregation**

In [18]:
probs_district = compute_district_average(probabilities, area)

In [19]:
probs_bc_district = compute_district_average(probabilities_bc, area)

In [20]:
# Build single xarray with merged unbiased/biased probabilities
probs_by_district = merge_un_biased_probs(
    probs_district.squeeze("time"),
    probs_bc_district.squeeze("time"),
    params,
    period_name,
)

**Dataframe formatting**

In [21]:
# Merge probabilities with triggers
probs_df, merged_df = merge_probabilities_triggers_dashboard(
    probs_by_district.drop_vars("time"), triggers_df, params, period_name
)

In [22]:
probs_df

,district,category,issue,index,prob,aggregation
0,Alto_Molocue,Mild,5,SPI ON,0.22,SPI 2
1,Alto_Molocue,Moderate,5,SPI ON,0.20,SPI 2
2,Alto_Molocue,Severe,5,SPI ON,0.16,SPI 2
3,Ancuabe,Mild,5,SPI ON,0.32,SPI 2
4,Ancuabe,Moderate,5,SPI ON,0.27,SPI 2
...,...,...,...,...,...,...
433,Zavala,Moderate,5,SPI ON,0.03,SPI 2
434,Zavala,Severe,5,SPI ON,0.00,SPI 2
435,Zumbu,Mild,5,SPI ON,0.29,SPI 2
436,Zumbu,Moderate,5,SPI ON,0.23,SPI 2


In [23]:
merged_df

,district,index,category,window,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set,season,date_ready,date_set
0,Cahora_Bassa,DRYSPELL AM,Severe,Window 2,11.0,12.0,0.15,0.31,NRT,0.13,0.25,2024-25,2024-11-01,2024-12-01
1,Cahora_Bassa,DRYSPELL FM,Mild,Window 2,9.0,10.0,0.02,0.28,NRT,0.18,0.18,2024-25,2024-09-01,2024-10-01
2,Cahora_Bassa,DRYSPELL FM,Severe,Window 2,9.0,10.0,0.10,0.30,NRT,0.05,0.08,2024-25,2024-09-01,2024-10-01
3,Cahora_Bassa,DRYSPELL JF,Mild,Window 1,8.0,9.0,0.05,0.06,NRT,0.22,0.30,2024-25,2024-08-01,2024-09-01
4,Cahora_Bassa,DRYSPELL JF,Moderate,Window 1,8.0,9.0,0.14,0.05,NRT,0.18,0.28,2024-25,2024-08-01,2024-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,Moamba,SPI NDJ,Moderate,Window 1,7.0,8.0,0.00,0.31,NRT,0.09,0.31,2024-25,2024-07-01,2024-08-01
418,Moamba,SPI NDJ,Severe,Window 1,7.0,8.0,0.19,0.18,NRT,0.09,0.22,2024-25,2024-07-01,2024-08-01
419,Moamba,SPI ON,Mild,Window 1,5.0,6.0,0.27,0.00,NRT,0.28,0.40,2024-25,2024-05-01,2024-06-01
420,Moamba,SPI ON,Moderate,Window 1,5.0,6.0,0.15,0.32,NRT,0.25,0.07,2024-25,2024-05-01,2024-06-01
